inertial frame bulk only

In [52]:
import numpy as np
from mlat.const import dirac

k1 = 1.
k2 = .5
m1 = 1.
m2 = 1.
qs = np.arange(0, 2 * np.pi, 0.1)

M = np.diag([m1, m1, m2, m2])
iM = np.linalg.inv(M)

r = dirac(2)
r11 = r.ket(0) * r.bra(0)
r22 = r.ket(1) * r.bra(1)
i = np.eye(2)


evals_all = []
evecs_all = []
for q in qs: 
    Q = np.exp(1.j * q)
    H = np.vstack([
        np.hstack([ (k1 + k2) * r22, -(k1 + k2 * Q.conj()) * r22]),
        np.hstack([-(k1 + k2 * Q) * r22, (k1 + k2) * r22]),
    ])
    # H = np.vstack([
    #     np.hstack([(k1 + k2), -(k1 + k2 * Q.conj()) ]),
    #     np.hstack([-(k1 + k2 * Q) , (k1 + k2) ]),
    # ])
    evals, evecs = np.linalg.eig(iM.dot(H))
    idcs = np.argsort(evals)
    evals, evecs = evals[idcs], evecs[idcs]
    evals_all.append(evals)
    evecs_all.append(evecs)
evals_all = np.array(evals_all)
evecs_all = np.array(evecs_all)
evals_all[:, 0].shape

# Plot

import plotly.graph_objects as go


fig = go.Figure(data=[go.Scatter(x=qs, y=np.sqrt(evals_all[:, i].real)) for i in [0, 1, 2, 3]])
fig.show()

inertial frame with edge

With coriolis force

In [60]:
import numpy as np
from mlat.const import dirac

k1 = 0.3
k2 = 1.
m1 = 1.
m2 = 1.
qs = np.arange(0, 2 * np.pi, 0.1)

M = np.diag([m1, m1, m2, m2])
iM = np.linalg.inv(M)

Omega = 1.

r = dirac(2)
r11 = r.ket(0) * r.bra(0)
r22 = r.ket(1) * r.bra(1)
i = np.eye(2)


evals_all = []
evecs_all = []
for q in qs: 
    Q = np.exp(1.j * q)
    H = np.vstack([
        np.hstack([ (k1 + k2) * i, -(k1 + k2 * Q.conj()) * i]),
        np.hstack([-(k1 + k2 * Q) * i, (k1 + k2) * i]),
    ])
    evals, evecs = np.linalg.eig(iM.dot(H))
    idcs = np.argsort(evals)
    evals, evecs = evals[idcs], evecs[idcs]
    evals_all.append(evals)
    evecs_all.append(evecs)
evals_all = np.array(evals_all)
evecs_all = np.array(evecs_all)
print(evals_all.shape)

# Plot

import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Scatter(x=qs, y=np.sqrt(evals_all[:, 0] - 1).real),
    go.Scatter(x=qs, y=np.sqrt(evals_all[:, 1].real)),
    go.Scatter(x=qs, y=np.sqrt(evals_all[:, 2] - 1).real),
    go.Scatter(x=qs, y=np.sqrt(evals_all[:, 3].real)),
])
fig.show()

(63, 4)
